In [6]:
import json
import re
from collections import Counter
from camel_tools.utils.normalize import normalize_alef_ar, normalize_teh_marbuta_ar, normalize_alef_maksura_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import pandas as pd
# تحميل قائمة Stop Words (يجب تحميلها أولاً باستخدام nltk)
nltk.download('stopwords')
arabic_stopwords = set(stopwords.words('arabic'))
# إضافة stopwords إضافية يدوية للتأكد من إزالة الكلمات الشائعة
additional_stopwords = {"الحا","البي","اي","مك","تف","بدل",'تمكنك', 'خدمة',"ال","مس","احد","بدون" ,"صك","اين","الا","الي","اخري","الح", "ضم",'يوجد', 'أن', 'يجب', 'أو', 'على', 'او', 'علي', 'ان', 'لا', 'في', 'من', 'عن', 'إلى', 'الى'}
arabic_stopwords.update(additional_stopwords)
print(f"Loaded {len(arabic_stopwords)} Arabic stopwords.")

Loaded 726 Arabic stopwords.


[nltk_data] Downloading package stopwords to C:\Users\Ahmed
[nltk_data]     fawzi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:

def load_data(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data

In [ ]:
class ArabicTextProcessor:
    def __init__(self):
        self.db = MorphologyDB.builtin_db()
        self.analyzer = Analyzer(self.db)

    def normalize_text(self, text):
        if not text or text == "- لا يوجد" or text.strip() == "":
            return ""
        text = dediac_ar(text)
        text = normalize_alef_ar(text)
        text = normalize_teh_marbuta_ar(text)
        text = normalize_alef_maksura_ar(text)
        text = re.sub(r'[^\w\s]', '', text)
        return text.strip()

    def tokenize_text(self, text):
        if not text:
            return []
        return simple_word_tokenize(text)

    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in arabic_stopwords]

    def lemmatize_text(self, tokens):
        lemmas = []
        for token in tokens:
            if token == "الدعاوي":
                token = "الدعوي"
            if token == "الدعوي":
                token = "دعوى"
            if token == "الخدمه":
                token = "خدمة"
            analyses = self.analyzer.analyze(token)
            if analyses and isinstance(analyses[0], dict):
                lemma = analyses[0].get('lemma', token)
                lemmas.append(lemma)
            else:
                lemmas.append(token)
        return lemmas

    def preprocess_text(self, text):
        if not text:
            return []
        normalized_text = self.normalize_text(text)
        tokens = self.tokenize_text(normalized_text)
        tokens = self.remove_stopwords(tokens)
        tokens = self.lemmatize_text(tokens)
        tokens = self.remove_stopwords(tokens)
        return tokens

    def extract_keywords_tfidf(self, documents, top_n=5):
        if not documents:
            return []
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(documents)
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray()
        keywords = []
        for doc_idx in range(len(documents)):
            scores = tfidf_scores[doc_idx]
            top_indices = scores.argsort()[-top_n:][::-1]
            top_keywords = [feature_names[idx] for idx in top_indices if scores[idx] > 0]
            keywords.append(top_keywords)
        return keywords


In [ ]:
class ServiceDataProcessor:
    def __init__(self):
        self.text_processor = ArabicTextProcessor()

    def process_service_data(self, service_data, category):
        processed_data = {
            "category": category,
            "service_name": self.text_processor.preprocess_text(service_data.get("service_name", "")),
            "description": self.text_processor.preprocess_text(service_data.get("description", "")),
            "keywords": []
        }
        text_for_keywords = " ".join([
            " ".join(processed_data["service_name"]),
            " ".join(processed_data["description"])
        ])
        return processed_data, text_for_keywords

    def build_assistant(self, raw_data):
        processed_data = []
        all_texts = []
        for service in raw_data:
            category = service.get("category", "")
            processed_service, text_for_keywords = self.process_service_data(service, category)
            processed_data.append(processed_service)
            all_texts.append(text_for_keywords)
        keywords_list = self.text_processor.extract_keywords_tfidf(all_texts)
        for idx, processed_service in enumerate(processed_data):
            processed_service["keywords"] = keywords_list[idx]
        return processed_data

    def combine_and_process_all(self, raw_data):
        """دمج ومعالجة الحقول category, service_name, description فقط."""
        full_text_parts = []
        for service in raw_data:
            service_text = " ".join([
                service.get("category", ""),
                service.get("service_name", ""),
                service.get("description", "")
            ]).replace("...","")
            full_text_parts.append(service_text)
        
        full_text = " ".join(full_text_parts)
        full_text_pre = self.text_processor.preprocess_text(full_text)
        full_text_pre_str = " ".join(full_text_pre)
        keywords = self.text_processor.extract_keywords_tfidf([full_text_pre_str])[0]
        
        return {
            "full_text": full_text,
            "full_text_pre": full_text_pre,
            "keywords": keywords
        }


In [10]:

def main():
    file_path = "questions_with_answers.json"
    raw_data = load_data(file_path)
    processor = ServiceDataProcessor()
    result = processor.combine_and_process_all(raw_data)
    print(json.dumps(result, ensure_ascii=False, indent=2))
    #with open("combined_processed_data.json", "w", encoding="utf-8") as f:
        #json.dump(result, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    main()

{
  "full_text": "التموين تفعيل بطاقة تموين تُمكّنك هذه الخدمة من تفعيل بطاقتك التموينية. التوثيق تحرير إقرار بالشطب (عن نفسه) تُمكّنك هذه الخدمة من تحرير إقرار بالشطب. التوثيق تحرير إقرار بعدم وجود تعديلات علي البي... تُمكّنك هذه الخدمة من تحرير إقرار قبول بيانات مساحية. التوثيق تحرير إقرار رسمي (عن نفسه) تُمكّنك هذه الخدمة من تحرير إقرار رسمي من مكانك. التوثيق تحرير إقرار تصحيح موثق (عن نفسه) تُمكّنك هذه الخدمة من تحرير إقرار تصحيح مُحرَّر موثق. التوثيق تحرير إقرار بالشطب (بصفة) تُمكّنك هذه الخدمة من تحرير إقرار بالشطب التوثيق تحرير إقرار تصحيح موثق (بصفة) تُمكّنك هذه الخدمة من تحرير إقرار تصحيح مُحرَّر موثق. التوثيق تحرير إقرار بعدم وجود تعديلات علي البي... تُمكّنك هذه الخدمة من تحرير إقرار قبول بيانات مساحية. التوثيق تحرير إقرار رسمي (بصفة) تُمكّنك هذه الخدمة من تحرير إقرار رسمي من مكانك. التوثيق تحرير عقد بيع مركبة تُمكّنك هذه الخدمة من توثيق عقد بيع أى من سياراتك السجل التجارى طلب تجديد سجل تجارى تُمكّنك هذه الخدمة من طلب تجديد سجل تجاري منتهٍ. السجل التجارى طلب شهادة سلبية الشها